In [14]:
import folium
import geojson
from branca.element import Figure
import geopandas as gpd
import pandas as pd

### Leer geosjon

In [15]:
gdf_lineas = gpd.read_file('metrobus_lineas.geojson',encoding='utf-8')

In [16]:
gdf_estaciones = gpd.read_file('metrobus_estaciones.geojson',encoding='utf-8')

In [17]:
gdf_estaciones.head()

,SISTEMA,NOMBRE,LINEA,EST,CVE_EST,CVE_EOD17,TIPO,ALCALDIAS,AÑO,geometry
0,Metrobús,Terminal 1 - Aeropuerto,04,None,None,None,None,None,NaN,POINT Z (-99.08367 19.43531 0.00000)
1,Metrobús,Terminal 2 - Aeropuerto,04,None,None,None,None,None,NaN,POINT Z (-99.07725 19.42148 0.00000)
2,Metrobús,Delegación Cuauhtémoc,04,02,MB0402,11121,Intermedia,Cuauhtémoc,2012.0,POINT Z (-99.15242 19.44259 0.00000)
3,Metrobús,Delegación Cuauhtémoc,04,02,MB0402,11121,Intermedia,Cuauhtémoc,2012.0,POINT Z (-99.15245 19.44214 0.00000)
4,Metrobús,México Tenochtitlan,04,None,None,None,None,None,NaN,POINT Z (-99.15327 19.43903 0.00000)


### Generar Mapa Base con Folium

In [18]:
m = folium.Map(
    location=[19.392668, -99.136829],
    #tiles="cartodbpositron",
    zoom_start=12.2,
    #width=600, height=400,
    tiles=None,
)

In [19]:
tile_layer = folium.TileLayer(
    control=False,
)
tile_layer.add_to(m)

In [20]:
fig = Figure(width=600, height=500)
fig.add_child(m)

### Pintar las estaciones y la rutas de acuerdo a la propiedad “LINEA”, crear checkbox y popups.


In [21]:
color_lines = {'1':'purple','2':'red','3':'black','4':'green','5':'blue','6':'orange','7':'cadetblue'}
lineas_grupos={}
for i in range(7): # Por cada linea, crear un grupo y una capa para el mapa
    temp_group = folium.FeatureGroup(name="Línea {}".format(i+1)).add_to(m)
    lineas_grupos[str(i+1)]=temp_group

In [22]:
lineas_grupos

{'1': <folium.map.FeatureGroup at 0x26002baa610>,
 '2': <folium.map.FeatureGroup at 0x26002baa340>,
 '3': <folium.map.FeatureGroup at 0x26002baaa60>,
 '4': <folium.map.FeatureGroup at 0x26002baa370>,
 '5': <folium.map.FeatureGroup at 0x26002baa2e0>,
 '6': <folium.map.FeatureGroup at 0x26002baa880>,
 '7': <folium.map.FeatureGroup at 0x26002baaaf0>}

In [23]:
style_function = lambda x: {'color': color_lines[x['properties']['LINEA']]}

for i in range(7): # Iterar por cada linea

    linea_df = gdf_lineas[gdf_lineas.LINEA==str(i+1)]


    lineas_grupos[str(i+1)].add_child(
        folium.GeoJson(linea_df, style_function=style_function, popup=folium.GeoJsonPopup(fields=['LINEA','RUTA','TRAMO']))
    )#.add_to(m)

    temp_estaciones = gdf_estaciones[gdf_estaciones.LINEA==str(0)+str(i+1)].reset_index(drop=True)

    # Iterar por cada estacion de cada linea
    for j in range(len(temp_estaciones)):
        html = '''
            <b>LINEA: </b>{linea_est}<br>
            <b>ESTACION: </b>{nombre}
            '''.format(nombre=temp_estaciones['NOMBRE'][j],linea_est=i+1)
            
        lineas_grupos[str(i+1)].add_child(folium.Marker([temp_estaciones.geometry[j].y,temp_estaciones.geometry[j].x],
                #Make color/style changes here
                icon = folium.Icon(color=color_lines[temp_estaciones.LINEA[j][1]],icon='bus',prefix= 'fa',icon_color='white',),
                popup = folium.Popup(html, parse_html=False, max_width=1000),
                ))#.add_to(m)
            
    

In [24]:
folium.map.LayerControl('topleft', collapsed= False).add_to(m)
m

### Mostrar una leyenda o una ventanita con los colores y a que línea corresponden.


In [25]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Líneas Metrobus</title>
 
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Líneas del Metrobus</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:purple;opacity:0.7;'></span>Línea 1</li>
    <li><span style='background:red;opacity:0.7;'></span>Línea 2</li>
    <li><span style='background:black;opacity:0.7;'></span>Línea 3</li>
    <li><span style='background:green;opacity:0.7;'></span>Línea 4</li>
    <li><span style='background:blue;opacity:0.7;'></span>Línea 5</li>
    <li><span style='background:orange;opacity:0.7;'></span>Línea 6</li>
    <li><span style='background:cadetblue;opacity:0.7;'></span>Línea 7</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)

In [26]:
m.save('lineas_metrobus.html')